### Запуск Spark

In [1]:
from pyspark.sql import SparkSession
from pyspark.sql import Row
from pyspark.sql.functions import *
import pandas as pd
import datetime
import os


spark = SparkSession \
            .builder \
            .config("spark.master", "local") \
            .config('spark.executor.cores', '8')\
            .config('spark.executor.memory', '2g')\
            .config('spark.driver.memory', '2g')\
            .config('spark.dynamicAllocation.minExecutors', '4')\
            .config("spark.jars.packages", "org.apache.hadoop:hadoop-aws:3.3.4") \
            .config("spark.jars.packages", "ru.yandex.clickhouse:clickhouse-jdbc:0.3.2") \
            .appName("spark_halltape") \
    .getOrCreate()


hadoop_conf = spark._jsc.hadoopConfiguration()
hadoop_conf.set("fs.s3a.impl", "org.apache.hadoop.fs.s3a.S3AFileSystem")
hadoop_conf.set("fs.s3a.access.key", os.environ["MINIO_ROOT_USER"])
hadoop_conf.set("fs.s3a.secret.key", os.environ["MINIO_ROOT_PASSWORD"])
hadoop_conf.set("fs.s3a.endpoint", "http://minio:9000")
hadoop_conf.set("fs.s3a.connection.ssl.enabled", "false")
hadoop_conf.set("fs.s3a.path.style.access", "true")
hadoop_conf.set("fs.s3a.aws.credentials.provider", "org.apache.hadoop.fs.s3a.SimpleAWSCredentialsProvider")


print("Активные Spark сессии:", 'http://localhost:4040/jobs')

:: loading settings :: url = jar:file:/opt/conda/lib/python3.11/site-packages/pyspark/jars/ivy-2.5.1.jar!/org/apache/ivy/core/settings/ivysettings.xml


Ivy Default Cache set to: /home/jovyan/.ivy2/cache
The jars for the packages stored in: /home/jovyan/.ivy2/jars
ru.yandex.clickhouse#clickhouse-jdbc added as a dependency
:: resolving dependencies :: org.apache.spark#spark-submit-parent-19603aba-507b-4fbc-8b45-84baf8cdfbe4;1.0
	confs: [default]
	found ru.yandex.clickhouse#clickhouse-jdbc;0.3.2 in central
	found com.clickhouse#clickhouse-http-client;0.3.2 in central
	found com.clickhouse#clickhouse-client;0.3.2 in central
	found org.lz4#lz4-java;1.8.0 in central
	found com.google.code.gson#gson;2.8.8 in central
	found org.apache.httpcomponents#httpclient;4.5.13 in central
	found org.apache.httpcomponents#httpcore;4.4.13 in central
	found commons-logging#commons-logging;1.2 in central
	found commons-codec#commons-codec;1.11 in central
	found org.apache.httpcomponents#httpmime;4.5.13 in central
:: resolution report :: resolve 355ms :: artifacts dl 15ms
	:: modules in use:
	com.clickhouse#clickhouse-client;0.3.2 from central in [default]
	

Активные Spark сессии: http://localhost:4040/jobs


In [2]:
# Создадим dataframe словарь
# Путь к файлу в S3
s3_path = "s3a://prod/api/earthquake/*.json"
s3_path_regions = "s3a://prod/jdbc/regions/*.parquet"

# Чтение JSON-файла
df = spark.read.json(s3_path)
regions = spark.read.parquet(s3_path_regions)

# # Покажем несколько строк
# df.show(truncate=False)


flattened = df.selectExpr("explode(features) as feature") \
    .select(
        col("feature.id").alias("id"),
        from_unixtime(col("feature.properties.time") / 1000).alias("time"),
        col("time").cast("date").alias("load_date"),
        trim(split(col("feature.properties.place"), ",").getItem(0)).alias("place"),
        trim(split(col("feature.properties.place"), ",").getItem(1)).alias("initial_region"), 
        col("feature.properties.mag").alias("magnitude"),
        col("feature.properties.felt").alias("felt"),
        col("feature.properties.tsunami").alias("tsunami"),
        col("feature.properties.url").alias("url"),
        col("feature.geometry.coordinates")[0].alias("longitude"),
        col("feature.geometry.coordinates")[1].alias("latitude"),
        col("feature.geometry.coordinates")[2].alias("depth"),
        md5(lower(trim(col("feature.properties.place")))).alias("place_hash")
    )


flattened.alias("f")\
    .join(regions.alias("r"), "place_hash", "left")\
    .where("initial_region IS NULL") \
    .drop('initial_region', 'place_hash') \
    .show(truncate=False)

# felt - Сколько людей сообщили, что ощутили землетрясение (в данном случае 1 человек)
# tsunami - Флаг цунами: 1 — возможно, 0 — нет
# url - Ссылка на страницу события на сайте USGS


25/05/17 21:14:58 WARN FileStreamSink: Assume no metadata directory. Error while looking for metadata directory in the path: s3a://prod/api/earthquake/*.json.
java.lang.RuntimeException: java.lang.ClassNotFoundException: Class org.apache.hadoop.fs.s3a.S3AFileSystem not found
	at org.apache.hadoop.conf.Configuration.getClass(Configuration.java:2688)
	at org.apache.hadoop.fs.FileSystem.getFileSystemClass(FileSystem.java:3431)
	at org.apache.hadoop.fs.FileSystem.createFileSystem(FileSystem.java:3466)
	at org.apache.hadoop.fs.FileSystem.access$300(FileSystem.java:174)
	at org.apache.hadoop.fs.FileSystem$Cache.getInternal(FileSystem.java:3574)
	at org.apache.hadoop.fs.FileSystem$Cache.get(FileSystem.java:3521)
	at org.apache.hadoop.fs.FileSystem.get(FileSystem.java:540)
	at org.apache.hadoop.fs.Path.getFileSystem(Path.java:365)
	at org.apache.spark.sql.execution.streaming.FileStreamSink$.hasMetadata(FileStreamSink.scala:53)
	at org.apache.spark.sql.execution.datasources.DataSource.resolveRe

Py4JJavaError: An error occurred while calling o40.json.
: java.lang.RuntimeException: java.lang.ClassNotFoundException: Class org.apache.hadoop.fs.s3a.S3AFileSystem not found
	at org.apache.hadoop.conf.Configuration.getClass(Configuration.java:2688)
	at org.apache.hadoop.fs.FileSystem.getFileSystemClass(FileSystem.java:3431)
	at org.apache.hadoop.fs.FileSystem.createFileSystem(FileSystem.java:3466)
	at org.apache.hadoop.fs.FileSystem.access$300(FileSystem.java:174)
	at org.apache.hadoop.fs.FileSystem$Cache.getInternal(FileSystem.java:3574)
	at org.apache.hadoop.fs.FileSystem$Cache.get(FileSystem.java:3521)
	at org.apache.hadoop.fs.FileSystem.get(FileSystem.java:540)
	at org.apache.hadoop.fs.Path.getFileSystem(Path.java:365)
	at org.apache.spark.sql.execution.datasources.DataSource$.$anonfun$checkAndGlobPathIfNecessary$1(DataSource.scala:724)
	at scala.collection.immutable.List.map(List.scala:293)
	at org.apache.spark.sql.execution.datasources.DataSource$.checkAndGlobPathIfNecessary(Da

In [ ]:
ch_max_df = spark.read \
    .format("jdbc") \
    .option("url", 'jdbc:clickhouse://clickhouse:8123/default',) \
    .option("user", "admin") \
    .option("password", "admin") \
    .option("dbtable", f"(SELECT MAX(load_date) as max_date FROM enriched_earthquakes)") \
    .option("driver", "com.clickhouse.jdbc.ClickHouseDriver") \
    .load()

max_date = ch_max_df.collect()[0]["max_date"].strftime("%Y-%m-%d")

if max_date == '1970-01-01':
    predicate = "1=1"
else:
    predicate = f"load_date > '{max_date}'"

predicate

In [ ]:
spark.stop()

### Сборка витрины данных

In [ ]:
PATH = '/home/jovyan/source'

def debit_card(date):
    card = spark.read.csv(f"{PATH}/card.csv", header=True, sep=";")\
                .where(f''' load_date = "{date}" ''')
    
    status_card = spark.read.csv(f"{PATH}/Status_card.csv", header=True, sep=";")\
                        .where(f''' load_date = "{date}"  ''')
    
    transactions = spark.read.csv(f"{PATH}/transactions.csv", header=True, sep=";")\
                        .where(f''' load_date = "{date}"  ''')
    
    
    first_trx = transactions.groupBy('card_num')\
                            .agg(F.min('transaction_datetime').alias('transaction_datetime'))
    
    
    first_trx_info = first_trx.join(transactions, ['card_num','transaction_datetime'], 'inner')
    
    dt_trx = first_trx_info.select('card_num', 'amount', '*')
    df_st = status_card.select('card_num', 'card_num_md5', 'status')
    
    result_df = dt_trx\
                    .join(df_st, "card_num", "inner")\
                    .join(card, "card_num_md5", "right")\
                    .drop('card_num_md5')
    
    final_df = result_df\
                    .where(''' card_num IS NOT NULL ''')\
                    .groupBy('card_num',
                             'transaction_datetime',
                             'status',
                             'card_order_dt',
                             'url',
                             'cookie')\
                    .agg(F.max('amount').alias('amt'))
    
    
    datamart = final_df\
                    .select('card_order_dt',
                            'card_num',
                            'cookie',
                            'url',
                            'amt',
                            'status')\
                    .withColumn('transaction_level',
                                    F.when(F.col('amt') > 300, True).otherwise(False))\
                    .withColumn('status_flag',
                                    F.when(F.col('status') == "выдана", True).otherwise(False))\
                    .withColumn('partition_date', F.lit(date).cast('string').alias('partition_date'))\
                    .withColumn('load_date', F.lit(date).cast('string').alias('load_date'))\
                    .drop('amt', 'status')
    
    datamart.write.mode("append").partitionBy('partition_date').csv('data_lake/debit_card', header=True)



start = '2024-07-01'
end = '2024-07-10'

for dt in pd.date_range(start, end):
    date = str(dt)[:10]
    print(date)
    debit_card(date)